In [148]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [149]:
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.offline as pyo
import plotly.express as px
pyo.init_notebook_mode(connected=True)
import utils as u

In [150]:
"""
Plain old dataset:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)

Enhanced with impressions:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)
"""

'\nPlain old dataset:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n\nEnhanced with impressions:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n'

In [151]:
"""
These destinations are selected based on their query pool sizes, and how representative
they are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)

9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),
482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),
9D883ED995259EA3E715122397FE0EF9 has query pool of size 2
"""
# selected_destinations = ["9FF55", "482E9", "9D883"]

'\nThese destinations are selected based on their query pool sizes, and how representative\nthey are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)\n\n9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),\n482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),\n9D883ED995259EA3E715122397FE0EF9 has query pool of size 2\n'

In [152]:

# def extra_df_prep(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.assign(
#         destination=df.apply(
#             lambda row: row.destination[0:5],
#             axis=1,
#         )
#     )
#     global selected_destinations
#     return df[df.destination.isin(selected_destinations)]


In [153]:
# unaugmented path
# path = "ray/criteo/bias_varying_epoch_size"

In [154]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [155]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [156]:

# u.plot_budget_consumption_lines(
#     path,
#     epoch_size=7,
#     by_destination=True,
#     extra_df_prep=extra_df_prep
# )

In [157]:
# u.save_data(path)

In [158]:
# augmented impressions path
# path = "ray/criteo/augmented_bias_varying_epoch_size"

In [159]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [160]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [161]:

# u.plot_budget_consumption_lines(
#     path,
#     epoch_size=7,
#     by_destination=True,
#     extra_df_prep=extra_df_prep
# )

In [162]:
# augmented convesrions path
path = "ray/criteo/augmented_conversions_bias_varying_epoch_size"

In [163]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [164]:
u.save_data(path)

In [165]:
df = pd.read_csv("../data/criteo/criteo_query_pool_augmented_conversions.csv")
df.head()

,partner_id,user_id,filter,product_category3,count,conversion_timestamp,epsilon,aggregatable_cap_value,key
0,C13DEAF04388B219AC04416DF3DA0ED8,4CA3BDF6304103161BEFC3DEAE5D0C2A,NaN,FE31DD5AD3E857EBD4BA2C4FDF732ED4,1.0,1596439563,0.757429,5,0
1,5DC3ED9F13993DF6471EBE25BD88BD24,F0D52A53241B145050B5ED8C55DD4B5B,NaN,8A30BF8BD2B3E9D44D2938D42C387D2E,1.0,1596439718,0.740381,5,0
2,6BFFDE668C5BC3938C875D3D0940E699,DC248B6867334C450AFDE6853A599E1E,NaN,F6BFE2644C2435F720C79B3EE10D9E42,1.0,1596439787,0.511686,5,0
3,9D883ED995259EA3E715122397FE0EF9,304E5E4A0F9F200B5A1CE22238AE1DE3,NaN,844A857A93A322847D10FA733714E3A6,4.0,1596439926,0.511686,5,0
4,319A2412BDB0EF669733053640B80112,79178DA7BA27DA4E64C4841D14DE747C,NaN,FAE775DB79B7E40FB3E581B2755783A4,1.0,1596439969,0.511686,5,8


In [166]:
true_output = df.groupby(["partner_id", "key"])["count"].sum()

In [167]:
print(true_output.shape, true_output[true_output > 0].shape)

(921,) (921,)


In [168]:
impressions = pd.read_csv("../data/criteo/criteo_query_pool_impressions.csv")

In [169]:
columns = ["partner_id", "user_id", "product_category3", "count", "conversion_timestamp", "epsilon", "key"]
x = df.merge(impressions, how="left", on=["partner_id", "user_id"], suffixes=("", "impressions"))
synthetic_conversions = x.loc[x.click_timestamp.isna()][columns]
real_conversions = x.loc[~x.click_timestamp.isna()][columns]

synthetic_counts = synthetic_conversions.groupby(["partner_id", "key"]).user_id.count().to_dict()
real_counts = real_conversions.groupby(["partner_id", "key"]).user_id.count().to_dict()

In [170]:
# Looks like somehow, the reports can end up fully populated
# by synthetic conversions

completely_synthetic_queries = []
for key in synthetic_counts.keys():
    if key not in real_counts:
        completely_synthetic_queries.append(key)

completely_synthetic_queries

[]

In [171]:
rmsres = pd.read_csv("../logs/ray/criteo/augmented_conversions_bias_varying_epoch_size/rmsres.csv")
rmsres.head()

,destination,queries_rmsres,baseline,initial_budget,workload_size,num_days_per_epoch,num_days_attribution_window
0,01E57DAC509AE48685D6219DE7718BA8,0.023343,ipa,1,1000,7,30
1,01E57DAC509AE48685D6219DE7718BA8,NaN,ipa,1,1000,7,30
2,01E57DAC509AE48685D6219DE7718BA8,NaN,ipa,1,1000,7,30
3,01E57DAC509AE48685D6219DE7718BA8,NaN,ipa,1,1000,7,30
4,0225D81C97298F6FD63C8868EE8E2228,0.020843,ipa,1,1000,7,30


In [172]:
# and the reports made up of only synthetic conversion records
# end up being the ones with na RMSREs.
for baseline, rdf in rmsres.loc[(rmsres.baseline != "ipa") & (rmsres.queries_rmsres.isna())].groupby(["baseline"]):
    print(f"__{baseline[0]}__")
    for partner, pdf in rdf.groupby(["destination"]):
        print(f"{partner[0]}:", pdf.shape[0])

__cookiemonster__
A57EF8EBFE4E6EF71F0DC4877B186793: 2
F762E4949B07C39AEA4F2D26849B1893: 5
__cookiemonster_base__
A57EF8EBFE4E6EF71F0DC4877B186793: 2
F762E4949B07C39AEA4F2D26849B1893: 5


In [173]:
for benchmark in range(1, 15):
    print(f"number of queries (by destination) with RMSREs >= {benchmark}")
    for baseline, rdf in rmsres.loc[(rmsres.baseline != "ipa") & (rmsres.queries_rmsres >= benchmark)].groupby(["baseline"]):
        print(f"\t{baseline[0]}")
        for partner, pdf in rdf.groupby(["destination"]):
            print(f"\t{partner[0]}:", pdf.shape[0])

number of queries (by destination) with RMSREs >= 1
	cookiemonster
	1BB44E9DB5ABBD9F9A64B475291DC555: 2
	319A2412BDB0EF669733053640B80112: 8
	6CCBF9BF3B87306C95BD5CD15F4CD324: 1
	8C8BBC051B051F9424483D0ADEFAB81B: 11
	95F6B7BA0BA90E5D5DF1CC063390BA51: 3
	98594F28C91322F1DAF20DF86F3F6D8A: 2
	9D9E93D1D461D7BAE47FB67EC0E01B62: 34
	A3B6CDB413E8C3A945248C364022DCB7: 1
	A57EF8EBFE4E6EF71F0DC4877B186793: 6
	F762E4949B07C39AEA4F2D26849B1893: 1
	cookiemonster_base
	1BB44E9DB5ABBD9F9A64B475291DC555: 2
	319A2412BDB0EF669733053640B80112: 8
	6CCBF9BF3B87306C95BD5CD15F4CD324: 1
	8C8BBC051B051F9424483D0ADEFAB81B: 11
	95F6B7BA0BA90E5D5DF1CC063390BA51: 3
	98594F28C91322F1DAF20DF86F3F6D8A: 2
	9D9E93D1D461D7BAE47FB67EC0E01B62: 34
	A3B6CDB413E8C3A945248C364022DCB7: 1
	A57EF8EBFE4E6EF71F0DC4877B186793: 6
	F762E4949B07C39AEA4F2D26849B1893: 1
number of queries (by destination) with RMSREs >= 2
	cookiemonster
	1BB44E9DB5ABBD9F9A64B475291DC555: 1
	319A2412BDB0EF669733053640B80112: 4
	6CCBF9BF3B87306C95BD5CD15F4